- [Weighted Random Recommendation (ДЗ 1)](#WeightedRandomRecommendation)
- [Baselines+ItemItem (ДЗ 2)](#Baselines)
- [Metrics (Итоги)](#metrics)

In [1]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k 

from sklearn.model_selection import GridSearchCV

    
from metrics import precision_at_k, recall_at_k

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation <a id='WeightedRandomRecommendation'></a>

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    recs = np.random.choice(item_weights['item_id'], p=item_weights['weight'], size=n, replace=False)
    
    return recs.tolist()
    

In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [7]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


In [8]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

Wall time: 4.39 s


In [9]:
item_weights = data.groupby('item_id')['user_id'].count().reset_index()
item_weights.sort_values('user_id', ascending=False, inplace=True)
total_count = item_weights['user_id'].sum()
item_weights['weight'] = item_weights['user_id'] / total_count
item_weights.head()

,item_id,user_id,weight
35054,1082185,27362,0.011416
56233,6534178,18364,0.007662
29195,1029743,13455,0.005614
25333,995242,11397,0.004755
37719,1106523,9175,0.003828


In [10]:
%%time
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(item_weights, n=5))

Wall time: 2.21 s


# Модели из вебинара

### Popularity-based recommendation

In [11]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [12]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 216 ms


,user_id,actual,random_recommendation,weighted_random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1131815, 868389, 1023693, 5979408, 9400399]","[847790, 989985, 1888133, 878996, 2405637]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[889766, 1006397, 10149261, 2060222, 9487342]","[1104584, 1015682, 1017363, 894524, 1054030]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### Item-Item

In [13]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [14]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [15]:
data_train['item_id'].isin(top_5000)

0           True
1           True
2           True
3           True
4           True
           ...  
2282320     True
2282321    False
2282322     True
2282323     True
2282324     True
Name: item_id, Length: 2278490, dtype: bool

In [16]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [17]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()



In [18]:
user_item_matrix.shape

(2499, 5001)

In [19]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [20]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 1.11 s


In [21]:
%%time

result['itemitem'] = result['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=None, 
                                                recalculate_user=True)
                                                             ])

Wall time: 71.2 ms


### Косинусное сходство и CosineRecommender

In [22]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 1.04 s


In [23]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 85 ms


In [24]:
result.head(5)

,user_id,actual,random_recommendation,weighted_random_recommendation,popular_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1131815, 868389, 1023693, 5979408, 9400399]","[847790, 989985, 1888133, 878996, 2405637]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[889766, 1006397, 10149261, 2060222, 9487342]","[1104584, 1015682, 1017363, 894524, 1054030]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1137554, 13115652, 983260, 1123150, 893687]","[1100872, 933637, 1092588, 834826, 896444]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1581104, 1027371, 1115107, 1114593, 9654995]","[1118012, 5568721, 15741531, 1139819, 1014732]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 6666, 1127831, 961554]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1775272, 822735, 963029, 1116250, 5872425]","[826419, 855488, 949620, 1018740, 1134882]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 6666, 1098066, 826249]"


###  TF-IDF взвешивание и TFIDFRecommender

In [25]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 1.23 s


In [26]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 1.21 s


In [27]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

Wall time: 92.8 ms


### Фильтрация + K = 1 (Item-item)

In [28]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 820 ms


In [29]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[6666]], 
                                    recalculate_user=True)])

Wall time: 77.6 ms


## Задание 2. Улучшение бейзлайнов и ItemItem <a id='Baselines'></a>

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

### Random recommendation / Weighted random recommendation для top5000

In [30]:
%%time

items = data_train.item_id.unique()

result['random_recommendation_top5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

Wall time: 345 ms


In [31]:
item_weights = data_train[data_train['item_id'] != 6666].groupby('item_id')['user_id'].count().reset_index()
item_weights.sort_values('user_id', ascending=False, inplace=True)
total_count = item_weights['user_id'].sum()
item_weights['weight'] = item_weights['user_id'] / total_count
item_weights.head()

,item_id,user_id,weight
3407,1082185,26127,0.018035
4345,6534178,17545,0.012111
2756,1029743,12737,0.008792
2307,995242,10943,0.007554
3687,1106523,8722,0.006021


In [32]:
result['weighted_random_recommendation_top5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(item_weights, n=5))

### ItemItemRecommender

Grid Search у меня не получилось реализовать, пробегусь обычным циклом, т.к. 1гиперпараметр

In [33]:
%%time
k_search = result.iloc[:, 0:2]
for K in range(1,11):
    model = ItemItemRecommender(K=K, num_threads=4) # K - кол-во билжайших соседей


    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=False)


    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                            N=5, # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)
    
    k_search['itemitem' + str(K)] = k_search['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=None, 
                                                recalculate_user=True)
                                                             ])


Wall time: 31.1 s


In [34]:
results_search = {}

for name_col in k_search.columns[1:]:
    results_search[name_col] = round(k_search.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)
total_search = pd.DataFrame.from_dict(results_search, orient='index', columns=['precision_at_k'])
total_search.sort_values(by='precision_at_k', ascending=False).head() 

,precision_at_k
actual,1.0000
itemitem1,0.1923
itemitem2,0.1920
itemitem3,0.1861
itemitem12,0.1535


Лучшим получился ItemItemRecomender c K = 1

In [35]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 1.16 s


In [36]:
result['itemitem_k1'] = result['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=None, 
                                                recalculate_user=True)
                                                             ])

# Metrics <a id='metrics'></a>

In [37]:
results = {}
for name_col in result.columns[1:]:
    results[name_col] = round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)

In [38]:
total = pd.DataFrame.from_dict(results, orient='index', columns=['precision_at_k'])
total.sort_values(by='precision_at_k', ascending=False)

,precision_at_k
actual,1.0000
own_purchases,0.2199
itemitem_k1,0.1923
popular_recommendation,0.1552
tfidf,0.1390
itemitem,0.1368
cosine,0.1329
weighted_random_recommendation_top5000,0.0290
weighted_random_recommendation,0.0196
random_recommendation_top5000,0.0051
